In [ ]:
%pip install selenium

---

## 구글 시트 불러오기

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
# 수정된 부분

In [4]:
# Google 스프레드시트 API 인증 범위 설정
scope = ['https://spreadsheets.google.com/feeds']

# 서비스 계정 JSON 키 파일 경로 설정
json_key_path = "key.json"	

# 인증 정보 로드
credential = ServiceAccountCredentials.from_json_keyfile_name(json_key_path, scope)
gc = gspread.authorize(credential)

# 스프레드시트 URL 설정
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

# URL을 통해 스프레드시트 열기
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
sheet_action = doc.worksheet("행위별 Category")

# 데이터 프레임으로 변환하기
df_action = pd.DataFrame(sheet_action.get_all_values())

# 첫 번째 행을 컬럼명으로 설정하고 나머지 데이터로 변환
df_action.rename(columns=df_action.iloc[0], inplace=True)
df_action.drop(df_action.index[0], inplace=True)

In [5]:
df_action

,보험 청구 Category,세부 청구 항목 (키워드),URL (tripletclover.com),코드
1,1. 청구 기초 및 기본진료,"구내염처치, 단순염증 처치",https://tripletclover.com/%ea%b5%ac%eb%82%b4%e...,"K05.22, K12.1, K05.32, K05.00, K05.18, K12.0, ..."
2,1. 청구 기초 및 기본진료,기본진료 종류 및 상병,https://tripletclover.com/%ea%b8%b0%eb%b3%b8%e...,"K01.12, K05.13, K02.8, K03.01, K07.69, K07.62,..."
3,1. 청구 기초 및 기본진료,기본진료시 항생제 처방 상병,https://tripletclover.com/%ea%b8%b0%eb%b3%b8-%...,"K01.12, K05.13, K07.69, K07.62, K07.66, K07.65..."
4,1. 청구 기초 및 기본진료,상세불명 vs 기타 명시된 상병명 차이,https://tripletclover.com/%ec%83%81%ec%84%b8%e...,"K02.9, K02.5, K02.3, K02.0, K02.8, K02.1, K02.2"
5,1. 청구 기초 및 기본진료,전달마취,https://tripletclover.com/%ec%a0%84%eb%8b%ac%e...,상병명 없음
...,...,...,...,...
107,8. 턱관절,악관절 탈구 비관혈적 정복술,https://tripletclover.com/%ec%95%85%ea%b4%80%e...,
108,8. 턱관절,악관절고착해소술,https://tripletclover.com/%ec%95%85%ea%b4%80%e...,
109,8. 턱관절,"악관절자극요법 (단순, 전기, 복합)",https://tripletclover.com/%ec%95%85%ea%b4%80%e...,
110,8. 턱관절,측두하악장애 분석검사,https://tripletclover.com/%ec%b8%a1%eb%91%90%e...,


In [9]:
df_action
urls = df_action['\x08URL  (tripletclover.com)'].tolist()

---
## 크롤링

In [10]:
import re
import requests
from bs4 import BeautifulSoup
import json  # 파일 저장을 위한 모듈

def crawling_code(urls):
    not_found_urls = []  # 상병명을 찾지 못한 URL들을 저장할 리스트
    extracted_codes_list = []  # 각 URL에서 추출된 코드를 저장할 리스트

    for url in urls:
        # 페이지 소스를 가져옴
        response = requests.get(url)
        html = response.text

        # BeautifulSoup으로 HTML 파싱
        soup = BeautifulSoup(html, 'html.parser')

        # id 속성에 "상병명"이 포함된 h2 태그 찾기 ("적용-가능-상병명-목록" 등도 포함되도록)
        start_section = soup.find('h2', id=lambda x: '상병명' in x if x else False)

        if start_section:
            # 알파벳숫자.숫자 패턴을 찾기 위한 정규 표현식
            pattern = re.compile(r'[A-Za-z]+\d+\.\d+')

            # wp-block-list 클래스의 모든 ul 태그 찾기
            wp_block_lists = start_section.find_all_next('ul', class_='wp-block-list')

            # p 태그와 ul 태그에서 패턴에 맞는 텍스트 추출
            extracted_codes = set()

            # ul 태그 내의 li 태그들에서 추출
            for wp_block_list in wp_block_lists:
                for li in wp_block_list.find_all('li'):
                    match = pattern.search(li.text)
                    if match:
                        extracted_codes.add(match.group())

            # p 태그에서도 같은 패턴을 추출
            for p_tag in start_section.find_all_next('p'):
                match = pattern.search(p_tag.text)
                if match:
                    extracted_codes.add(match.group())

            # 추출된 코드 정렬
            sorted_codes = sorted(extracted_codes)

            # 추출된 코드 출력
            print("Extracted codes:", sorted_codes)
            print("Number of extracted codes:", len(sorted_codes))
            extracted_codes_list.append(", ".join(sorted_codes) if sorted_codes else "코드 없음")

        else:
            # 상병명을 찾지 못한 URL을 저장
            print(f"No section with '상병명' found in {url}")
            not_found_urls.append(url)
            extracted_codes_list.append("상병명 없음")  # 못찾으면 코드 없음으로 기록

    # 상병명을 찾지 못한 URL을 파일로 저장
    if not_found_urls:
        with open('not_found_urls.json', 'w') as f:
            json.dump(not_found_urls, f, ensure_ascii=False, indent=4)

    # 상병명을 찾지 못한 URL과 크롤링된 코드 리스트 반환
    return not_found_urls, extracted_codes_list


In [ ]:
not_found, extracted_codes = crawling_code(urls)

# 결과 출력
print("URLs where '상병명' was not found:", not_found)
print("Extracted codes for each URL:", extracted_codes)

Extracted codes: ['B00.2', 'B37.0', 'K05.00', 'K05.10', 'K05.18', 'K05.19', 'K05.22', 'K05.32', 'K12.0', 'K12.1', 'K13.1', 'S00.5']
Number of extracted codes: 12
Extracted codes: ['J01.00', 'K01.1', 'K01.10', 'K01.11', 'K01.12', 'K01.13', 'K01.14', 'K01.15', 'K01.163', 'K01.173', 'K02.0', 'K02.1', 'K02.2', 'K02.3', 'K02.5', 'K02.8', 'K03.00', 'K03.01', 'K03.10', 'K03.18', 'K03.2', 'K03.80', 'K04.00', 'K04.7', 'K05.1', 'K05.10', 'K05.11', 'K05.12', 'K05.13', 'K05.18', 'K05.2', 'K05.20', 'K05.21', 'K05.22', 'K05.28', 'K05.29', 'K05.30', 'K05.31', 'K07.6', 'K07.60', 'K07.61', 'K07.62', 'K07.63', 'K07.64', 'K07.65', 'K07.66', 'K07.68', 'K07.69']
Number of extracted codes: 48
Extracted codes: ['J01.00', 'K01.10', 'K01.11', 'K01.12', 'K01.13', 'K01.14', 'K01.15', 'K01.163', 'K01.173', 'K04.7', 'K05.12', 'K05.13', 'K05.18', 'K05.20', 'K05.21', 'K05.22', 'K05.28', 'K05.29', 'K05.30', 'K05.31', 'K07.60', 'K07.61', 'K07.62', 'K07.63', 'K07.64', 'K07.65', 'K07.66', 'K07.68', 'K07.69']
Number of e

In [ ]:
not_found

In [ ]:
extracted_codes

In [ ]:
# '코드' 열에 상위 10개의 크롤링 결과만 추가
df_action.loc[:10, '코드'] = extracted_codes  # 인덱스 0~9까지에 해당하는 10개 행에만 추가

df_action.head(10)

### '코드' 열 전체를 스프레드시트에 업데이트

In [ ]:
# 데이터프레임의 전체 행에 맞는 셀 범위 설정
num_rows = len(df_action)  # 데이터프레임의 총 행 수
cell_range = f'D2:D{num_rows + 1}'  # D2부터 전체 행에 해당하는 범위 지정

# '코드' 열의 전체 값을 추출
code_column_values = df_action['코드'].tolist()

# 스프레드시트의 '코드' 열을 업데이트
sheet_action.update(cell_range, [[code] for code in code_column_values])